In [62]:
%load_ext autoreload
%autoreload 2
import logging
import multiprocessing
import os
import shutil
import sys
import time
import types

import numpy as np
import pandas as pd

from linajea.config import TrackingConfig
import linajea.evaluation

from linajea.training import train
import linajea.config
import linajea.process_blockwise
import linajea.utils
from blockwise.candidate_graph_blockwise import extract_edges_blockwise

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import zarr
z = zarr.open ( '/groups/funke/home/xuz2/Alice_demo_2.zarr' , 'r' )

fragments = z['Fragments'][:]
raw = z['Raw'][:]
#track = z['gt_trackimage']
print(z.tree(level=2))

/
 ├── Fragment_stats
 │   ├── Position
 │   ├── Volume
 │   └── id
 ├── Fragments (3, 15, 25, 25) uint64
 ├── Merge_tree
 │   ├── Merge
 │   └── Scoring
 └── Raw (4, 3, 15, 25, 25) float64


In [63]:
import sys
sys.path.append('/groups/funke/home/xuz2/fatepredict_experiments')

In [2]:

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory:", cwd)

Current working directory: /groups/funke/home/xuz2/fatepredict_experiments/example


In [70]:
config_file = 'config.toml'
config = TrackingConfig.from_file(config_file)

In [71]:
extract_edges_blockwise(config)

cabdidategraph_extract_edges ▶:   0%|          | 0/500 [00:00<?, ?blocks/s]

Block cabdidategraph_extract_edges/0 with read ROI [-3:0, -2:2, -2:2, -2:2] (3, 4, 4, 4) and write ROI [-2:0, -2:2, -2:2, -2:2] (2, 4, 4, 4) failed in worker 822 with AssertionError('Requested roi [-3:0, -2:2, -2:2, -2:2] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
Block cabdidategraph_extract_edges/1 with read ROI [-3:0, -2:2, -2:2, 2:6] (3, 4, 4, 4) and write ROI [-2:0, -2:2, -2:2, 2:6] (2, 4, 4, 4) failed in worker 822 with AssertionError('Requested roi [-3:0, -2:2, -2:2, 2:6] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
Block cabdidategraph_extract_edges/5 with read ROI [-3:0, -2:2, -2:2, 6:10] (3, 4, 4, 4) and write ROI [-2:0, -2:2, -2:2, 6:10] (2, 4, 4, 4) failed in worker 822 with AssertionError('Requested roi [-3:0, -2:2, -2:2, 6:10] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
Block cabdidategraph_extract_edges/15 with read ROI [-3:0, -2:2, -2:2, 10:14


Execution Summary
-----------------

  Task cabdidategraph_extract_edges:

    num blocks : 500
    completed ✔: 0 (skipped 0)
    failed    ✗: 0
    orphaned  ∅: 0

    Failed Blocks:

      block 0 in worker 822 with exception AssertionError('Requested roi [-3:0, -2:2, -2:2, -2:2] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
      block 1 in worker 822 with exception AssertionError('Requested roi [-3:0, -2:2, -2:2, 2:6] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
      block 5 in worker 822 with exception AssertionError('Requested roi [-3:0, -2:2, -2:2, 6:10] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
      block 15 in worker 822 with exception AssertionError('Requested roi [-3:0, -2:2, -2:2, 10:14] (3, 4, 4, 4) is not contained in this array [0:3, 0:15, 0:25, 0:25] (3, 15, 25, 25).')
      block 2 in worker 823 with exception AssertionError('Requested roi

In [28]:
data = config.inference_data.data_source
graph_provider = linajea.utils.CandidateDatabase(data.db_name,
                                                config.general.db_host,
                                                mode='r+')

In [49]:
import daisy
extract_roi = daisy.Roi(offset=data.roi.offset,
                            shape=data.roi.shape)

In [66]:
block_write_roi = daisy.Roi(
        (0,)*4,
        daisy.Coordinate(config.extract.block_size))

    # set thresgold
max_edge_move_th = max(config.extract.edge_move_threshold.values())
pos_context = daisy.Coordinate((0,) + (max_edge_move_th,)*3)
neg_context = daisy.Coordinate((1,) + (max_edge_move_th,)*3)


input_roi = extract_roi.grow(neg_context, pos_context)
block_read_roi = block_write_roi

In [67]:
block_read_roi

[0:2, 0:4, 0:4, 0:4] (2, 4, 4, 4)